In [1]:
#Part 2 III.2

In [2]:
#import the necessary libraries 
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns
import folium
import statsmodels.api as sm
import scipy.stats
import numpy as np
from math import sqrt
import statsmodels.formula.api as smf
import osmnx as ox

In [3]:
cdata = pd.read_csv("london_flows.csv")
cdata.head(10)

,station_origin,station_destination,flows,population,jobs,distance
0,Abbey Road,Bank and Monument,0,599,78549,8131.525097
1,Abbey Road,Beckton,1,599,442,8510.121774
2,Abbey Road,Blackwall,3,599,665,3775.448872
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220
4,Abbey Road,Canning Town,37,599,15428,2228.923167
5,Abbey Road,Crossharbour,1,599,1208,6686.475560
6,Abbey Road,Custom House,0,599,845,3824.855630
7,Abbey Road,Cutty Sark,2,599,1748,8503.898909
8,Abbey Road,Cyprus,7,599,850,6532.099618
9,Abbey Road,Devons Road,1,599,611,3958.324171


In [4]:
cdata.replace(0,0.0000001,inplace=True)

In [5]:
cdata.head(10)

,station_origin,station_destination,flows,population,jobs,distance
0,Abbey Road,Bank and Monument,1.000000e-07,599.0,78549.0,8131.525097
1,Abbey Road,Beckton,1.000000e+00,599.0,442.0,8510.121774
2,Abbey Road,Blackwall,3.000000e+00,599.0,665.0,3775.448872
3,Abbey Road,Canary Wharf,1.000000e+00,599.0,58772.0,5086.514220
4,Abbey Road,Canning Town,3.700000e+01,599.0,15428.0,2228.923167
5,Abbey Road,Crossharbour,1.000000e+00,599.0,1208.0,6686.475560
6,Abbey Road,Custom House,1.000000e-07,599.0,845.0,3824.855630
7,Abbey Road,Cutty Sark,2.000000e+00,599.0,1748.0,8503.898909
8,Abbey Road,Cyprus,7.000000e+00,599.0,850.0,6532.099618
9,Abbey Road,Devons Road,1.000000e+00,599.0,611.0,3958.324171


In [6]:
#1.unconstrain 

In [7]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

#take the variables and produce logarithms of them
x_variables = ["population", "jobs", "distance"]
log_x_vars = []
for x in x_variables:
    cdata[f"log_{x}"] = np.log(cdata[x])
    log_x_vars.append(f"log_{x}")

#create the formula
formula = 'flows ~ log_population + log_jobs + log_distance'

#run the regression
uncosim = smf.glm(formula = formula, 
                  data=cdata, 
                  family=sm.families.Poisson()).fit()

In [8]:
print(uncosim.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  flows   No. Observations:                61474
Model:                            GLM   Df Residuals:                    61470
Model Family:                 Poisson   Df Model:                            3
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.4618e+06
Date:                Sun, 01 May 2022   Deviance:                   2.7514e+06
Time:                        11:32:12   Pearson chi2:                 6.89e+06
No. Iterations:                    21                                         
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept        -10.2333      0.010  -1031.

In [9]:
#first assign the parameter values from the model to the appropriate variables
K = uncosim.params[0]
alpha = uncosim.params[1]
gamma = uncosim.params[2]
beta = -uncosim.params[3]

#now plug everything back into the Equation 6 model ... 
#be careful with the negative signing of the parameter beta
cdata["unconstrained"] = np.exp(K 
                                       + alpha*cdata["log_population"] 
                                       + gamma*cdata["log_jobs"] 
                                       - beta*cdata["log_distance"])

#or we can just extract the results from the actual poisson regression and apply them to the data
predictions = uncosim.get_prediction()
predictions_summary_frame = predictions.summary_frame()
cdata["fitted"] = predictions_summary_frame["mean"]

In [10]:
uncosim.mu

array([46.22141833,  0.61187156,  0.90492749, ...,  4.59052217,
        9.29918596, 11.84541264])

In [11]:
#round the numbers so that we don't get a half of a person
cdata["unconstrained"] = round(cdata["unconstrained"], 0)
#convert to integers
cdata["unconstrained"] = cdata["unconstrained"].astype(int)
#check that the sum of these estimates make sense
sum(cdata["unconstrained"])

1542327

In [12]:
import scipy.stats

def CalcRSqaured(observed, estimated):
    """Calculate the r^2 from a series of observed and estimated target values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""
    
    r, p = scipy.stats.pearsonr(observed, estimated)
    R2 = r **2
    
    return R2

CalcRSqaured(cdata["flows"], cdata["unconstrained"])

0.2658986114505707

In [13]:
# constrained 

In [14]:
cdatasubmat = pd.pivot_table(cdata, values ="flows", index="station_origin", columns = "station_destination",
                            aggfunc=np.sum, margins=True)
cdatasubmat

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.000000,5.990000e+02
Acton Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e-07,NaN,NaN,1.224000e+03
Acton Town,NaN,NaN,NaN,3.000000e+00,1.700000e+01,NaN,3.500000e+01,1.000000e-07,NaN,11.000000,...,7.700000e+01,3.000000e+00,6.000000e+00,9.000000,NaN,1.000000e-07,NaN,1.000000e-07,NaN,3.745000e+03
Aldgate,NaN,NaN,1.000000e-07,NaN,1.000000e-07,NaN,NaN,1.000000e-07,NaN,17.000000,...,1.000000e-07,NaN,4.000000e+00,8.000000,NaN,1.000000e-07,NaN,1.000000e-07,NaN,2.886000e+03
Aldgate East,NaN,NaN,2.000000e+00,1.000000e-07,NaN,NaN,1.000000e-07,1.000000e-07,NaN,20.000000,...,2.400000e+01,1.000000e-07,1.000000e-07,12.000000,NaN,1.000000e+00,NaN,1.000000e+00,NaN,3.172000e+03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,NaN,NaN,2.000000e+00,5.000000e+00,4.700000e+01,NaN,NaN,NaN,NaN,22.000000,...,2.000000e+00,NaN,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,4.868000e+03
Woodgrange Park,NaN,1.000000e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.300000e+02
Woodside Park,NaN,NaN,1.000000e+00,2.600000e+01,1.100000e+01,NaN,1.000000e-07,NaN,NaN,59.000000,...,1.000000e-07,NaN,1.000000e-07,NaN,NaN,NaN,NaN,NaN,NaN,3.093000e+03


In [15]:
cdatasubmat2 = cdata.pivot_table(values ="unconstrained", index="station_origin", columns = "station_destination",
                            aggfunc=np.sum, margins=True)
cdatasubmat2

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,189
Acton Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,344
Acton Town,NaN,NaN,NaN,29.0,29.0,NaN,4.0,2.0,NaN,30.0,...,24.0,3.0,10.0,8.0,NaN,4.0,NaN,4.0,NaN,4671
Aldgate,NaN,NaN,8.0,NaN,27.0,NaN,NaN,1.0,NaN,26.0,...,19.0,NaN,8.0,6.0,NaN,3.0,NaN,3.0,NaN,4035
Aldgate East,NaN,NaN,9.0,29.0,NaN,NaN,3.0,1.0,NaN,28.0,...,21.0,3.0,9.0,7.0,NaN,3.0,NaN,4.0,NaN,4426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,NaN,NaN,12.0,36.0,36.0,NaN,NaN,NaN,NaN,37.0,...,29.0,NaN,12.0,NaN,NaN,NaN,NaN,NaN,NaN,5596
Woodgrange Park,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54
Woodside Park,NaN,NaN,8.0,24.0,25.0,NaN,3.0,NaN,NaN,26.0,...,20.0,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,3750


In [16]:
#Production-constrained Model

In [17]:
#create the formula (the "-1" indicates no intercept in the regression model).
formula = 'flows ~ station_origin + log_jobs + log_distance-1'
#run a production constrained sim
prodSim = smf.glm(formula = formula, data=cdata, family=sm.families.Poisson()).fit()
#let's have a look at it's summary
print(prodSim.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  flows   No. Observations:                61474
Model:                            GLM   Df Residuals:                    61073
Model Family:                 Poisson   Df Model:                          400
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.2592e+06
Date:                Sun, 01 May 2022   Deviance:                   2.3462e+06
Time:                        11:36:15   Pearson chi2:                 4.76e+06
No. Iterations:                    22                                         
Covariance Type:            nonrobust                                         
                                                  coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------

In [18]:
#get the predictions
predictions = prodSim.get_prediction(cdata[["station_origin", "log_jobs", "log_distance"]])
predictions_summary_frame = predictions.summary_frame()
cdata["prosimFitted"] = round(predictions_summary_frame["mean"],0)
#now we can create pivot table to turn paired list into matrix (and compute the margins as well)
cdatamat3 = cdata.pivot_table(values ="prosimFitted", index="station_origin", columns = "station_destination",
                                    aggfunc=np.sum, margins=True)
cdatamat3

/opt/conda/envs/sds2021/lib/python3.9/site-packages/statsmodels/genmod/families/links.py:517: RuntimeWarning: overflow encountered in exp
  return np.exp(z)


station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,601.0
Acton Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,1227.0
Acton Town,NaN,NaN,NaN,23.0,23.0,NaN,4.0,1.0,NaN,24.0,...,19.0,3.0,8.0,7.0,NaN,3.0,NaN,3.0,NaN,3741.0
Aldgate,NaN,NaN,6.0,NaN,20.0,NaN,NaN,1.0,NaN,19.0,...,14.0,NaN,6.0,5.0,NaN,2.0,NaN,2.0,NaN,2888.0
Aldgate East,NaN,NaN,6.0,21.0,NaN,NaN,3.0,1.0,NaN,20.0,...,15.0,2.0,6.0,5.0,NaN,3.0,NaN,3.0,NaN,3176.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,NaN,NaN,11.0,31.0,32.0,NaN,NaN,NaN,NaN,32.0,...,25.0,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,4870.0
Woodgrange Park,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,529.0
Woodside Park,NaN,NaN,7.0,20.0,21.0,NaN,3.0,NaN,NaN,21.0,...,17.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,3099.0


In [19]:
cdatasubmat

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.000000,5.990000e+02
Acton Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e-07,NaN,NaN,1.224000e+03
Acton Town,NaN,NaN,NaN,3.000000e+00,1.700000e+01,NaN,3.500000e+01,1.000000e-07,NaN,11.000000,...,7.700000e+01,3.000000e+00,6.000000e+00,9.000000,NaN,1.000000e-07,NaN,1.000000e-07,NaN,3.745000e+03
Aldgate,NaN,NaN,1.000000e-07,NaN,1.000000e-07,NaN,NaN,1.000000e-07,NaN,17.000000,...,1.000000e-07,NaN,4.000000e+00,8.000000,NaN,1.000000e-07,NaN,1.000000e-07,NaN,2.886000e+03
Aldgate East,NaN,NaN,2.000000e+00,1.000000e-07,NaN,NaN,1.000000e-07,1.000000e-07,NaN,20.000000,...,2.400000e+01,1.000000e-07,1.000000e-07,12.000000,NaN,1.000000e+00,NaN,1.000000e+00,NaN,3.172000e+03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,NaN,NaN,2.000000e+00,5.000000e+00,4.700000e+01,NaN,NaN,NaN,NaN,22.000000,...,2.000000e+00,NaN,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,4.868000e+03
Woodgrange Park,NaN,1.000000e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.300000e+02
Woodside Park,NaN,NaN,1.000000e+00,2.600000e+01,1.100000e+01,NaN,1.000000e-07,NaN,NaN,59.000000,...,1.000000e-07,NaN,1.000000e-07,NaN,NaN,NaN,NaN,NaN,NaN,3.093000e+03


In [20]:
def CalcRSquared(observed, estimated):
    """Calculate the r^2 from a series of observed and estimated target values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""
    
    r, p = scipy.stats.pearsonr(observed, estimated)
    R2 = r **2
    
    return R2

CalcRSquared(cdata["flows"], cdata["prosimFitted"])

0.3399030531926003

In [21]:
#attraction 

In [22]:
#create the formula (the "-1" indicates no intercept in the regression model).
attr_form = 'flows ~ station_destination + log_population + log_distance'
#run a production constrained sim
attrSim = smf.glm(formula = attr_form, data=cdata, family=sm.families.Poisson()).fit()
#let's have a look at it's summary
print(attrSim.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  flows   No. Observations:                61474
Model:                            GLM   Df Residuals:                    61073
Model Family:                 Poisson   Df Model:                          400
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.3359e+06
Date:                Sun, 01 May 2022   Deviance:                   2.4995e+06
Time:                        11:41:31   Pearson chi2:                 5.60e+06
No. Iterations:                    22                                         
Covariance Type:            nonrobust                                         
                                                         coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------

In [23]:
#get the predictions
predictions = attrSim.get_prediction(cdata[["station_destination", "log_population", "log_distance"]])
predictions_summary_frame = predictions.summary_frame()
cdata["attrsimFitted"] = round(predictions_summary_frame["mean"],0)
#now we can create pivot table to turn paired list into matrix (and compute the margins as well)
cdatasubmat4 = cdata.pivot_table(values ="attrsimFitted", index="station_origin", columns = "station_destination",
                                    aggfunc=np.sum, margins=True)
cdatasubmat4

/opt/conda/envs/sds2021/lib/python3.9/site-packages/statsmodels/genmod/families/links.py:517: RuntimeWarning: overflow encountered in exp
  return np.exp(z)


station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0,360.0
Acton Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,635.0
Acton Town,NaN,NaN,NaN,27.0,26.0,NaN,3.0,1.0,NaN,26.0,...,22.0,3.0,7.0,7.0,NaN,3.0,NaN,3.0,NaN,4312.0
Aldgate,NaN,NaN,6.0,NaN,24.0,NaN,NaN,1.0,NaN,23.0,...,18.0,NaN,6.0,5.0,NaN,2.0,NaN,2.0,NaN,3715.0
Aldgate East,NaN,NaN,7.0,27.0,NaN,NaN,3.0,1.0,NaN,24.0,...,19.0,2.0,6.0,6.0,NaN,2.0,NaN,3.0,NaN,4068.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,NaN,NaN,9.0,33.0,32.0,NaN,NaN,NaN,NaN,31.0,...,27.0,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,5197.0
Woodgrange Park,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140.0
Woodside Park,NaN,NaN,6.0,23.0,22.0,NaN,3.0,NaN,NaN,22.0,...,19.0,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,3496.0


In [24]:
CalcRSquared(cdata["flows"], cdata["attrsimFitted"])

0.29698721376274523

In [25]:
# Double constrain

In [ ]:
import statsmodels.formula.api as smf
#create the formula (the "-1" indicates no intercept in the regression model).
dbl_form = 'flows ~ station_origin + station_destination + log_distance-1'
#run a doubly constrained sim
doubSim = smf.glm(formula = dbl_form, data=cdata, family=sm.families.Poisson()).fit()
#let's have a look at it's summary
print(doubSim.summary())

In [ ]:
#get the estimates
cdata["doubsimfitted"] = np.round(doubSim.mu)
#here's the matrix
cdatasubmat5 = cdata.pivot_table(values ="doubsimfitted", index="station_origin", columns = "station_destination",
                                    aggfunc=np.sum, margins=True)
cdatasubmat5

In [ ]:
CalcRSqaured(cdata["flows"],cdata["doubsimfitted"])